# Mobility Meeting Scheduler

#### Constants

In [164]:
SOLVER_TIME_LIMIT = 2000

MAXIMUM_FLIGHT_COST = 2500
MAXIMUM_USEFUL_TIME = 2628288

#### Import Data

In [165]:
import json
from datetime import datetime

# Import students information
file = open('../../data/students.json')

input = json.load(file)

DESTINATIONS = input['destinations']
MINIMUM_USEFUL_TIME = input['minimumTime']
STUDENTS_INFO = input['students']
N_STUDENTS = len(STUDENTS_INFO)

STUDENTS_ORIGINS = [DESTINATIONS.index(x) for x in [student["city"] for student in STUDENTS_INFO]]
STUDENTS_STOPS = [student["maxConnections"] for student in STUDENTS_INFO]
STUDENTS_DURATIONS = [student["maxDuration"] for student in STUDENTS_INFO]
STUDENTS_DEPARTURES = [int(round(datetime.strptime(student["earliestDeparture"], '%H:%M:%S').timestamp())) for student in STUDENTS_INFO]
STUDENTS_ARRIVALS = [int(round(datetime.strptime(student["latestArrival"], '%H:%M:%S').timestamp())) for student in STUDENTS_INFO]

STUDENS_AVAILABILITIES = []
for student in STUDENTS_INFO:
    studentIntervals = []
    for start, end in student["availability"]:
        studentIntervals.append(int(round(datetime.strptime(start, '%d/%m/%Y').timestamp())))
        studentIntervals.append(int(round(datetime.strptime(end, '%d/%m/%Y').timestamp())))
    STUDENS_AVAILABILITIES.append(studentIntervals)

N_MAX_INTERVALS = max([int(len(x) / 2) for x in STUDENS_AVAILABILITIES])

print("Imported", N_STUDENTS, "students,", len(DESTINATIONS), "destinations and the minimum time.")


# Import flights
file = open('../../data/flights.json')

FLIGHTS = json.load(file)

FLIGHTS_ORIGINS = [DESTINATIONS.index(x) for x in [flight["origin"] for flight in FLIGHTS]]
FLIGHTS_DESTINATIONS = [DESTINATIONS.index(x) for x in [flight["destination"] for flight in FLIGHTS]]

FLIGHTS_DEPARTURES = [int(round(datetime.strptime(flight["departure"], '%d/%m/%Y, %H:%M:%S ').timestamp())) for flight in FLIGHTS]
FLIGHTS_DEPARTURE_TIMES = [int(round(datetime.strptime(datetime.fromtimestamp(departure).time().isoformat(), '%H:%M:%S').timestamp())) for departure in FLIGHTS_DEPARTURES]

FLIGHTS_ARRIVALS = [int(round(datetime.strptime(flight["arrival"], '%d/%m/%Y, %H:%M:%S ').timestamp())) for flight in FLIGHTS]
FLIGHTS_ARRIVAL_TIMES = [int(round(datetime.strptime(datetime.fromtimestamp(arrival).time().isoformat(), '%H:%M:%S').timestamp())) for arrival in FLIGHTS_ARRIVALS]

FLIGHTS_DURATIONS = [flight["duration"] for flight in FLIGHTS]
FLIGHTS_COSTS = [int(flight["price"]) for flight in FLIGHTS]
FLIGHTS_STOPS = [flight["stops"] for flight in FLIGHTS]

print("Imported", len(FLIGHTS), "flights!")

Imported 3 students, 3 destinations and the minimum time.
Imported 113 flights!


#### Model

In [166]:
from docplex.cp.model import CpoModel
from docplex.cp.expression import INT_MIN, INT_MAX

model = CpoModel()

#### Variables

In [167]:
# Chosen Destination
Destination = model.integer_var(0, len(DESTINATIONS) - 1, "Destination")

# Indexes of the flights each student has to take
StudentsFlights = [model.integer_var_list(2, 0, len(FLIGHTS) - 1) for i in range(N_STUDENTS)]

# Student avaliability interval
StudentsAvailabilityIntervals = model.integer_var_list(N_STUDENTS, 0, N_MAX_INTERVALS, "Interval")

# Cost for each of the students
StudentsCosts = model.integer_var_list(N_STUDENTS, 0, MAXIMUM_FLIGHT_COST, "StudentCost")

# Total trip cost
TotalCost = model.integer_var(0, MAXIMUM_FLIGHT_COST * N_STUDENTS, "TotalCost")

# Useful time
UsefulTime = model.integer_var(0, MAXIMUM_USEFUL_TIME, "UsefulTime")

#### Constraints

In [168]:
for i in range(N_STUDENTS):
    Outgoing, Incoming = StudentsFlights[i]
    StudentOrigin = model.element(STUDENTS_ORIGINS, i)

    # Origin of the flights
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_ORIGINS, Outgoing) == StudentOrigin))
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_ORIGINS, Incoming) == Destination))

    # Destination of the flights
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_DESTINATIONS, Outgoing) == Destination))
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_DESTINATIONS, Incoming) == StudentOrigin))

    # Availability
    startAvailability = model.element(STUDENS_AVAILABILITIES[i], StudentsAvailabilityIntervals[i] * 2)
    endAvailability = model.element(STUDENS_AVAILABILITIES[i], StudentsAvailabilityIntervals[i] * 2 + 1)
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_DEPARTURES, Outgoing) >= startAvailability))
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_ARRIVALS, Incoming) <= endAvailability))

    # Outgoing arrival time must be before Incoming departure time
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_ARRIVALS, Outgoing) < model.element(FLIGHTS_DEPARTURES, Incoming)))

    # Earliest departure
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_DEPARTURE_TIMES, Outgoing) >= model.element(STUDENTS_DEPARTURES, i)))
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_DEPARTURE_TIMES, Incoming) >= model.element(STUDENTS_DEPARTURES, i)))

    # Latest arrival
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_ARRIVAL_TIMES, Outgoing) <= model.element(STUDENTS_ARRIVALS, i)))
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_ARRIVAL_TIMES, Incoming) <= model.element(STUDENTS_ARRIVALS, i)))

    # Maximum number of stops
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_STOPS, Outgoing) <= model.element(STUDENTS_STOPS, i)))
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_STOPS, Incoming) <= model.element(STUDENTS_STOPS, i)))

    # Maximum flight duration
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_DURATIONS, Outgoing) <= model.element(STUDENTS_DURATIONS, i)))
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_DURATIONS, Incoming) <= model.element(STUDENTS_DURATIONS, i)))

    # Student Cost
    model.add(model.conditional(StudentOrigin == Destination, 0, model.element(FLIGHTS_COSTS, StudentsFlights[i][0]) + model.element(FLIGHTS_COSTS, StudentsFlights[i][1])) == StudentsCosts[i])


# Minimum Useful Time
lastOutgoingTime = model.max([model.conditional(model.element(STUDENTS_ORIGINS, i) != Destination, model.element(FLIGHTS_ARRIVALS, StudentsFlights[i][0]), INT_MIN) for i in range(N_STUDENTS)])
firstIncomingTime = model.min([model.conditional(model.element(STUDENTS_ORIGINS, i) != Destination, model.element(FLIGHTS_DEPARTURES, StudentsFlights[i][1]), INT_MAX) for i in range(N_STUDENTS)])

model.add(UsefulTime == firstIncomingTime - lastOutgoingTime)

model.add(UsefulTime >= MINIMUM_USEFUL_TIME)

# Minimize total cost
model.add(TotalCost == model.sum(StudentsCosts))
model.add(model.minimize(TotalCost))

#### Solve

In [169]:
print("Solving model....")

solution = model.solve(TimeLimit = SOLVER_TIME_LIMIT)

Solving model....
 ! --------------------------------------------------- CP Optimizer 22.1.0.0 --
 ! Minimization problem - 15 variables, 51 constraints
 ! TimeLimit            = 2000
 ! Initial process time : 0.02s (0.02s extraction + 0.00s propagation)
 !  . Log search space  : 105.8 (before), 105.8 (after)
 !  . Memory usage      : 462.4 kB (before), 462.4 kB (after)
 ! Using parallel search with 8 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0         15                 -
 + New bound is 0
                        0         15    1   F        -
 + New bound is 170
                        8         11    1   F     0 != Interval_1
 + New bound is 171
 *           743       16  0.08s        1      (gap is 76.99%)
 *           615       23  0.08s        1      (gap is 72.20%)
 *           597       32  0.08s        1      (gap is 71.36%)
 *           585   

#### Solution

In [170]:
if solution:
    # solution.print_solution()
    print("Solution status: " + solution.get_solve_status())
    print("--> Destination: " + DESTINATIONS[solution[Destination]])
    print("--> Total Cost: " + str(solution[TotalCost]))
    print("--> Useful Time: " + str(solution[UsefulTime]))
    print("--> Students:")

    for i in range(N_STUDENTS):
        Outgoing, Incoming = StudentsFlights[i]

        print("    --> Student " + str(i + 1) + " departing from " + STUDENTS_INFO[i]["city"] + ":")

        if STUDENTS_ORIGINS[i] == solution[Destination]:
            print("       - Student need not to take any flights!")
        else:
            print("       -", list(FLIGHTS[solution[Outgoing]].items())[2:])
            print("       -", list(FLIGHTS[solution[Incoming]].items())[2:])

else:
    print("No solution found")

Solution status: Optimal
--> Destination: budapest
--> Total Cost: 422
--> Useful Time: 144300
--> Students:
    --> Student 1 departing from budapest:
       - Student need not to take any flights!
    --> Student 2 departing from zagreb:
       - [('departure', '06/05/2022, 14:10:00 '), ('arrival', '06/05/2022, 20:50:00 '), ('duration', 400), ('price', '105'), ('stops', 1)]
       - [('departure', '09/05/2022, 07:00:00 '), ('arrival', '09/05/2022, 11:00:00 '), ('duration', 240), ('price', '129'), ('stops', 1)]
    --> Student 3 departing from wien:
       - [('departure', '07/05/2022, 07:00:00 '), ('arrival', '07/05/2022, 14:55:00 '), ('duration', 475), ('price', '102'), ('stops', 1)]
       - [('departure', '09/05/2022, 09:15:00 '), ('arrival', '09/05/2022, 14:45:00 '), ('duration', 330), ('price', '86'), ('stops', 1)]
